In [1]:
import pandas as pd

In [2]:
file = pd.read_excel("Data.xlsx")

In [3]:
sales_table = file.loc[1:,:"Unnamed: 6"]
sales_table.columns = file.loc[0, :"Unnamed: 6"].to_list()
sales_table.set_index("SalesID", drop= True, inplace= True)

In [4]:
sale_profit = file.loc[1:, "Sale Profit":"Unnamed: 10"]
sale_profit.dropna(how= "all", inplace= True)
sale_profit.columns = file.loc[0, "Sale Profit":"Unnamed: 10"].to_list()
sale_profit["ProfitRatio"] = (sale_profit["ProfitRatio"] * 100).astype(int)

In [5]:
organization_table = file.loc[1:, "چارت سازمانی":"Unnamed: 16"]
organization_table.columns = file.loc[0, "چارت سازمانی":"Unnamed: 16"].to_list()
organization_table.dropna(how= "all", inplace= True)
organization_table.set_index("Id", drop= True, inplace= True)
organization_table.fillna("0", inplace= True)

In [6]:
sales_table

,OrderID,Customer,Product,Date,Quantity,UnitPrice
SalesID,,,,,,
1,1,C1,P1,1,2,100
2,1,C1,P2,1,4,150
3,2,C2,P2,1,5,150
4,3,C3,P4,1,3,550
5,4,C4,P3,1,1,300
6,4,C4,P6,1,6,150
7,4,C4,P4,1,6,550
8,5,C5,P2,1,3,150
9,5,C5,P1,1,6,100


In [7]:
sale_profit

,Product,ProfitRatio
1,P1,5
2,P2,25
3,P3,10
4,P4,20
5,P5,10


In [8]:
organization_table

,name,manager,Manager Id
Id,,,
1,Ken,0,0
2,Hugo,0,0
3,James,Carol,5
4,Mark,Morgan,13
5,Carol,Alex,12
6,David,Rose,21
7,Michael,Markos,11
8,Brad,Alex,12
9,Rob,Matt,15


In [9]:
import pyodbc as db
import db_config

In [10]:
server = db_config.server 
database = db_config.database
username = db_config.username
con_string = 'DRIVER={ODBC Driver 18 for SQL Server};SERVER=' + server\
     + ';DATABASE=' + database + ';Trusted_Connection=yes;UID=' + username\
     + ';TrustServerCertificate=yes'

In [11]:
with db.connect(con_string) as cnxn:
    cnxn.execute("""
                    IF OBJECT_ID(N'sale_table', N'U') IS NULL 
                    BEGIN   
                        CREATE TABLE sale_table (
                            SalesID INT, 
                            OrderID INT, 
                            Customer VARCHAR(10), 
                            Product VARCHAR(10), 
                            Date INT, 
                            Quantity INT, 
                            UnitPrice INT
                        );
                    END;
                """)
    cnxn.execute("""
                IF OBJECT_ID(N'sale_profit', N'U') IS NULL 
                BEGIN   
                    CREATE TABLE sale_profit (
                        ProductID INT, 
                        Product VARCHAR(10), 
                        ProfitRatio SMALLINT
                    );
                END;
            """)
    cnxn.execute("""
                IF OBJECT_ID(N'organ_chart', N'U') IS NULL 
                BEGIN   
                    CREATE TABLE organ_chart (
                        Id SMALLINT, 
                        Name VARCHAR(30), 
                        Manager VARCHAR(30),
                        Manager_ID SMALLINT
                    );
                END;
            """)
    cnxn.commit()
    for row in sales_table.itertuples():
        cnxn.execute(f"""INSERT INTO sale_table 
                     VALUES ({row[0]}, {row[1]}, '{row[2]}', '{row[3]}', {row[4]}, {row[5]}, {row[6]});""")
    cnxn.commit()
    for row in sale_profit.itertuples():
        cnxn.execute(f"""INSERT INTO sale_profit 
                     VALUES ({row[0]}, '{row[1]}', {row[2]});""")
    cnxn.commit()
    for row in organization_table.itertuples():
        cnxn.execute(f"""INSERT INTO organ_chart 
                     VALUES ({row[0]}, '{row[1]}', '{row[2]}', {row[3]});""")
    cnxn.commit()

In [16]:
with db.connect(con_string) as cnxn:
    total_sale = cnxn.execute("SELECT SUM(UnitPrice) FROM sale_table;").fetchall()[0][0]
    number_of_customers = cnxn.execute("SELECT COUNT(DISTINCT(Customer)) FROM sale_table;").fetchall()[0][0]
    product_sales = cnxn.execute("SELECT Product, COUNT(*) FROM sale_table GROUP BY Product ORDER BY Product ASC;").fetchall()
    #print(product_sales)
    good_customers = cnxn.execute("SELECT DISTINCT(Customer), SUM(Quantity * UnitPrice), COUNT(*), SUM(Quantity) FROM sale_table GROUP BY Customer HAVING SUM(Quantity * UnitPrice) > 1500 ORDER BY Customer ASC").fetchall()
    #print(good_customers)
    total_profit = cnxn.execute("""SELECT SUM(sale_table.Quantity * sale_table.UnitPrice * ISNULL(sale_profit.ProfitRatio,10) / 100), SUM(sale_table.Quantity * sale_table.UnitPrice * ISNULL(sale_profit.ProfitRatio,10)) / SUM(sale_table.Quantity * sale_table.UnitPrice)
                                    FROM sale_table
                                    FULL JOIN
                                    sale_profit
                                    ON sale_table.Product = sale_profit.Product
                                    """).fetchall()
    #print(total_profit)
    daily_customers = cnxn.execute("SELECT COUNT(*) FROM (SELECT DISTINCT Customer, Date FROM sale_table) AS customers_per_day").fetchall()[0][0]
    #print(daily_customers)
    #cnxn.execute("ALTER TABLE organ_chart ADD level SMALLINT, top_manager VARCHAR(30)")
    organ_chart = cnxn.execute("""WITH manager_hierarchy AS (
                    SELECT e.Name, e.Manager, CAST(e.Name AS VARCHAR(MAX)) AS top_manager, 1 AS level, e.Manager AS Manager
                    FROM organ_chart e
                    LEFT JOIN organ_chart m ON e.manager = m.Name
                    WHERE m.Name IS NULL

                    UNION ALL

                    SELECT e.Name, e.Manager, mh.top_manager, mh.level + 1, m.Manager
                    FROM organ_chart e
                    JOIN manager_hierarchy mh ON e.manager = mh.Name
                    JOIN organ_chart m ON e.manager = m.Name
                    )
                    SELECT Name, Manager, level, top_manager
                    FROM manager_hierarchy;""").fetchall()
    print(organ_chart)
    cnxn.commit()
    


[('Ken', '0', 1, 'Ken'), ('Hugo', '0', 1, 'Hugo'), ('Matt', 'Hugo', 2, 'Hugo'), ('Rob', 'Matt', 3, 'Hugo'), ('Morgan', 'Matt', 3, 'Hugo'), ('Mark', 'Morgan', 4, 'Hugo'), ('Jennifer', 'Morgan', 4, 'Hugo'), ('Fill', 'Morgan', 4, 'Hugo'), ('Eddie', 'Mark', 5, 'Hugo'), ('Daniel', 'Rob', 4, 'Hugo'), ('Rose', 'Rob', 4, 'Hugo'), ('Robert', 'Rob', 4, 'Hugo'), ('David', 'Rose', 5, 'Hugo'), ('Antoan', 'David', 6, 'Hugo'), ('Alex', 'Ken', 2, 'Ken'), ('Carol', 'Alex', 3, 'Ken'), ('Brad', 'Alex', 3, 'Ken'), ('Dylan', 'Alex', 3, 'Ken'), ('Oliver', 'Dylan', 4, 'Ken'), ('Ana', 'Dylan', 4, 'Ken'), ('Tom', 'Brad', 4, 'Ken'), ('James', 'Carol', 4, 'Ken'), ('Markos', 'Carol', 4, 'Ken'), ('Michael', 'Markos', 5, 'Ken'), ('Amanda', 'Markos', 5, 'Ken')]
